In [1]:
# Core libs
import pandas as pd
import numpy as np
import importlib

# Graphic libs
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
import seaborn as sns
pio.renderers.default = "browser"

# sklearn 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_absolute_percentage_error as LMAPE

# IA + blockchain = profit
import tensorflow as tf
from keras import optimizers
from keras.models import Model
from keras.layers import Input, Lambda, Dropout, BatchNormalization, Activation, Dense, LeakyReLU, Add
from keras.layers.merging import Add, Concatenate
from keras.utils import plot_model, to_categorical

2022-12-07 22:36:51.552672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 22:36:52.201451: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-07 22:36:53.347679: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-07 22:36:53.347795: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
raw_data_master = pd.read_excel('../data/xlsx/ord80p.xlsx', header = [0])
raw_column_names = raw_data_master.columns

In [6]:
raw_data_augment = raw_data_master.drop('ranva', axis = 1).copy()
raw_data_augment.insert(raw_data_augment.shape[1], 'Area code', np.zeros(raw_data_augment.shape[0], dtype=str))

In [11]:
fechas = raw_data_augment[raw_data_augment['Edad'] >= 1000]['Fecha entrega del Informe'].to_numpy()
edades = raw_data_augment[raw_data_augment['Edad'] >= 1000]['Edad'].to_numpy()

In [16]:
edades[1]

1931.0

In [21]:
np.round(((fechas[1] - pd.Timestamp(edades[1])).days)/365.0)

50.0

In [22]:
edades = raw_data_augment['Edad'].to_numpy()

for i, edad in enumerate(edades):
    if edad >= 1000:
        edades[i] = np.round(((fechas[1] - pd.Timestamp(edades[1])).days)/365.0)

raw_data_augment['Edad'] = edades

In [23]:
raw_data_augment['Edad'].unique()

array([ 15.,   5.,  16.,   1.,   0.,   2.,  12.,  30.,  48.,   6.,  73.,
        60.,   4.,  19.,  50.,  20.,   8.,  35.,  nan,  23.,  18.,  24.,
        72.,  52.,  17.,  36.,  38.,  10.,  46.,   3.,  11.,  41.,  22.,
        14.,  33., 365.,  27.,  40.,  21.,  39.,  25.,   7.,  59.,  61.,
        55.,   9.,  29.,  42.,  31.,  57.,  56.,  54.,  13.,  45.,  47.,
        37.,  32.,  34.,  53.,  69.,  26.,  49.,  43., 103.,  62.,  63.,
        68.,  70.,  64.,  65.,  58.,  44., 155.,  76.,  75.,  28.,  78.,
        80.,  82.,  51.,  74.,  67.,  85.,  84., 141.,  95., 100.,  77.,
        90.,  79., 137., 105.,  81., 126.,  71.,  66.,  88.,  87., 106.,
        91., 108.,  92.,  83., 114.,  89., 430., 101., 200.,  86.,  99.,
       112., 110., 119.,  93.,  94.,  97., 122., 118., 303.,  96., 117.,
       124., 199., 107., 138., 109., 102., 306., 595., 164.,  98., 111.,
       154., 115., 260., 136., 357.])

In [28]:
area_data = dict()
ac_dict = dict()

for dep_id, dep in enumerate(raw_data_master['Departamento'].unique()):
    dep_select = raw_data_master[raw_data_master['Departamento'] == dep]

    for prod_id, prov in enumerate(dep_select['Provincia'].unique()):
        prov_select = dep_select[dep_select['Provincia'] == prov]

        for dist_id, dist in enumerate(prov_select['Distrito'].unique()):

            query = raw_data_master[(raw_data_master['Departamento'] == dep  ) & 
                                    (raw_data_master['Provincia']    == prov ) &
                                    (raw_data_master['Distrito']     == dist )]
            area_code = str(dep_id) + '-' +str(prod_id) + '-' +str(dist_id)
            code_list = raw_data_augment['Area code'].to_numpy()
            code_list[query.index] = area_code
            raw_data_augment['Area code'] = code_list

            area_data[(dep,prov,dist)] = []
            ac_dict[area_code] = (dep,prov,dist)

In [34]:
raw_data_augment

,Fecha entrega del Informe,Tipo de vía,Piso,Departamento,Provincia,Distrito,Número de estacionamiento,Depósitos,Latitud (Decimal),Longitud (Decimal),...,Posición,Número de frentes,Edad,Elevador,Estado de conservación,Método Representado,Área Terreno,Área Construcción,Valor comercial (USD),Area code
0,2020-02-17,3.0,NaN,Madre de Dios,Tambopata,Tambopata,NaN,NaN,-12.597282,-69.179768,...,NaN,NaN,15.0,NaN,Bueno,Costos o reposición (directo),75.00,189.50,169607,0-0-0
1,2018-06-21,3.0,NaN,Lima,Lima,Santiago de Surco,0.0,0.0,-12.128921,-76.976432,...,NaN,NaN,5.0,NaN,Bueno,Comparación de mercado (directo),12.72,93.87,160437,1-0-0
2,2019-05-08,2.0,NaN,Lima,Lima,San Juan de Miraflores,0.0,0.0,-12.162320,-76.970215,...,NaN,NaN,16.0,NaN,Regular,Costos o reposición (directo),160.00,446.40,294567,1-0-1
3,2019-03-10,3.0,NaN,Lima,Lima,Surquillo,1.0,0.0,-12.112404,-77.014408,...,NaN,NaN,1.0,NaN,En construcción,Comparación de mercado (directo),3.94,85.56,124749,1-0-2
4,2021-01-17,1.0,NaN,Lambayeque,Lambayeque,Lambayeque,0.0,0.0,-6.699075,-79.901748,...,NaN,NaN,0.0,NaN,En proyecto,Costos o reposición (directo),60.00,27.58,23501,2-0-0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42599,2018-09-18,3.0,NaN,Callao,Prov. Const. del Callao,Callao,0.0,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Costos o reposición (directo),NaN,NaN,11217,6-0-1
42600,2019-09-13,1.0,NaN,Piura,Piura,Veintiseis de Octubre,0.0,0.0,-5.163182,-80.682388,...,NaN,NaN,0.0,NaN,En construcción,Costos o reposición (directo),60.00,27.58,25253,14-0-3
42601,2021-04-19,3.0,NaN,Lima,Lima,La Molina,2.0,0.0,-12.082234,-76.957287,...,NaN,NaN,12.0,NaN,Bueno,Comparación de mercado (directo),121.13,185.95,267706,1-0-11
42602,2016-05-29,1.0,NaN,Lima,Lima,Villa El Salvador,0.0,0.0,-12.224233,-76.935714,...,NaN,NaN,16.0,NaN,NaN,Comparación de mercado (directo),140.00,280.00,95480,1-0-21


In [ ]:
raw_data_augment[raw_data_augment['Area code'] == '10-0-0']

In [13]:
TO_AUGMENT = [
    'Número de estacionamiento',
    'Depósitos',
    'Edad'
]

for augment_dim in TO_AUGMENT:
    for area_code in raw_data_augment['Area code'].unique():
        query = raw_data_augment[raw_data_augment['Area code'] == area_code]

        to_augment_array = np.array([val.replace(',','') if isinstance(val, str) else val for val in query[augment_dim].to_list()], dtype=np.float32)
        to_augment_array = np.nan_to_num(to_augment_array, nan = -1)

        augment_mean = np.round(np.mean(to_augment_array[to_augment_array != -1])) if to_augment_array[to_augment_array != -1].shape[0] > 0 else 0
        to_augment_array[to_augment_array == -1] = augment_mean

        full_array = raw_data_augment[augment_dim].to_numpy()
        full_array[query.index] = to_augment_array

        raw_data_augment[augment_dim] = full_array

raw_data_augment['Área Terreno'] = raw_data_augment['Área Terreno'].fillna(0.0)
raw_data_augment['Área Construcción'] = raw_data_augment['Área Construcción'].fillna(0.0)

In [26]:
#raw_data_augment[raw_data_augment['Area code'] == '1-0-11']
raw_data_augment

,Fecha entrega del Informe,Tipo de vía,Piso,Departamento,Provincia,Distrito,Número de estacionamiento,Depósitos,Latitud (Decimal),Longitud (Decimal),...,Posición,Número de frentes,Edad,Elevador,Estado de conservación,Método Representado,Área Terreno,Área Construcción,Valor comercial (USD),Area code
27,43780,2.0,NaN,Lima,Lima,La Molina,0.0,0.0,-12.099151,-76.936925,...,NaN,NaN,18.0,NaN,Bueno,Costos o reposición (directo),311.90,378.78,490752,1-0-11
37,43501,1.0,NaN,Lima,Lima,La Molina,0.0,0.0,-12.071605,-76.937411,...,NaN,NaN,8.0,NaN,Bueno,Costos o reposición (directo),468.36,139.05,851367,1-0-11
38,42687,2.0,NaN,Lima,Lima,La Molina,1.0,0.0,-12.082800,-76.908600,...,NaN,NaN,5.0,NaN,NaN,Comparación de mercado (directo),309.68,250.22,537973,1-0-11
75,43601,1.0,NaN,Lima,Lima,La Molina,4.0,1.0,-12.090006,-76.930280,...,NaN,NaN,365.0,NaN,NaN,Costos o reposición (directo),982.00,554.88,1561343,1-0-11
88,43446,1.0,NaN,Lima,Lima,La Molina,0.0,0.0,-12.078218,-76.919327,...,NaN,NaN,18.0,NaN,Muy bueno,Comparación de mercado (directo),963.37,669.56,1509886,1-0-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42438,43172,1.0,NaN,Lima,Lima,La Molina,0.0,0.0,-12.092648,-76.953519,...,NaN,NaN,10.0,NaN,Bueno,Comparación de mercado (directo),40.58,124.08,193735,1-0-11
42482,43761,1.0,NaN,Lima,Lima,La Molina,0.0,0.0,-12.087835,-76.952148,...,NaN,NaN,8.0,NaN,Muy bueno,Costos o reposición (directo),600.00,481.40,864442,1-0-11
42543,43241,1.0,NaN,Lima,Lima,La Molina,1.0,0.0,-12.074715,-76.932612,...,NaN,NaN,12.0,NaN,Bueno,Comparación de mercado (directo),540.00,405.52,831316,1-0-11
42548,43121,1.0,NaN,Lima,Lima,La Molina,0.0,0.0,-12.073204,-76.941636,...,NaN,NaN,19.0,NaN,NaN,Costos o reposición (directo),220.00,0.0,290400,1-0-11


In [27]:
raw_data_augment.insert(raw_data_augment.shape[1], 'Precio promedio m2 (Construcción)', np.zeros(raw_data_augment.shape[0], dtype=str))
raw_data_augment.insert(raw_data_augment.shape[1], 'Precio promedio m2 (Terreno)', np.zeros(raw_data_augment.shape[0], dtype=str))

In [28]:
for area_code in raw_data_augment['Area code'].unique():
    query = raw_data_augment[raw_data_augment['Area code'] == area_code]

    build_area_array = np.array([val.replace(',','') if isinstance(val, str) else val for val in query['Área Construcción'].to_list()], dtype=np.float64)
    plot_area_array = np.array([val.replace(',','') if isinstance(val, str) else val for val in query['Área Terreno'].to_list()], dtype=np.float64)
    
    price_array = np.array([val.replace(',','') if isinstance(val, str) else val for val in query['Valor comercial (USD)'].to_list()], dtype=np.float64)

    build_area_array = np.nan_to_num(build_area_array, nan=-1)
    build_area_array[build_area_array == 0] = -1
    plot_area_array = np.nan_to_num(plot_area_array, nan=-1)
    plot_area_array[plot_area_array == 0] = -1

    sqrt_m_bval = price_array/build_area_array
    sqrt_m_bval = sqrt_m_bval[sqrt_m_bval >= 0]

    sqrt_m_pval = price_array/plot_area_array
    sqrt_m_pval = sqrt_m_pval[sqrt_m_pval >= 0]

    avg_bval_list = raw_data_augment['Precio promedio m2 (Construcción)'].to_numpy()
    avg_pval_list = raw_data_augment['Precio promedio m2 (Terreno)'].to_numpy()

    avg_bval_list[query.index] = np.round(np.mean(sqrt_m_bval)) if sqrt_m_bval.shape[0] > 0 else 0
    avg_pval_list[query.index] = np.round(np.mean(sqrt_m_pval)) if sqrt_m_bval.shape[0] > 0 else 0
    raw_data_augment['Precio promedio m2 (Construcción)'] = avg_bval_list
    raw_data_augment['Precio promedio m2 (Terreno)'] = avg_pval_list

    #code_list[query.index] = area_code
    #raw_data_augment['Area code'] = code_list

In [29]:
raw_data_augment[raw_data_augment['Area code'] == '1-0-1']

,Fecha entrega del Informe,Tipo de vía,Piso,Departamento,Provincia,Distrito,Número de estacionamiento,Depósitos,Latitud (Decimal),Longitud (Decimal),...,Edad,Elevador,Estado de conservación,Método Representado,Área Terreno,Área Construcción,Valor comercial (USD),Area code,Precio promedio m2 (Construcción),Precio promedio m2 (Terreno)
2,43593,2.0,NaN,Lima,Lima,San Juan de Miraflores,0.0,0.0,-12.162320,-76.970215,...,16.0,NaN,Regular,Costos o reposición (directo),160.00,446.40,294567,1-0-1,1594.0,2873.0
21,44269,NaN,NaN,Lima,Lima,San Juan de Miraflores,0.0,0.0,-12.173931,-76.960601,...,8.0,NaN,Regular,Comparación de mercado (directo),18.42,80.08,64864,1-0-1,1594.0,2873.0
171,43685,1.0,NaN,Lima,Lima,San Juan de Miraflores,0.0,0.0,-12.174948,-76.956416,...,15.0,NaN,Regular,Costos o reposición (directo),160.40,111.86,108718,1-0-1,1594.0,2873.0
185,43138,3.0,NaN,Lima,Lima,San Juan de Miraflores,0.0,0.0,-12.165190,-76.971207,...,1.0,NaN,Bueno,Comparación de mercado (directo),22.02,99.95,128566,1-0-1,1594.0,2873.0
220,43667,1.0,NaN,Lima,Lima,San Juan de Miraflores,0.0,0.0,-12.174945,-76.980480,...,11.0,NaN,Bueno,Costos o reposición (directo),200.00,426.16,298835,1-0-1,1594.0,2873.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41944,43410,1.0,NaN,Lima,Lima,San Juan de Miraflores,0.0,0.0,-12.151129,-76.982696,...,40.0,NaN,Bueno,Comparación de mercado (directo),17.75,61.73,55557,1-0-1,1594.0,2873.0
41959,43814,NaN,NaN,Lima,Lima,San Juan de Miraflores,0.0,0.0,-12.140085,-76.977485,...,26.0,NaN,Bueno,Costos o reposición (directo),187.20,562.00,247032,1-0-1,1594.0,2873.0
42067,43843,1.0,NaN,Lima,Lima,San Juan de Miraflores,0.0,0.0,-12.182605,-76.966259,...,5.0,NaN,Bueno,Costos o reposición (directo),197.80,197.80,201456,1-0-1,1594.0,2873.0
42243,43466,2.0,NaN,Lima,Lima,San Juan de Miraflores,0.0,0.0,-12.155715,-76.973670,...,41.0,NaN,Bueno,Comparación de mercado (directo),160.00,199.00,209558,1-0-1,1594.0,2873.0


In [ ]:
ordinal_data = raw_data_master[raw_data_master['Provincia'] == 'Lima'].copy()

for column in raw_column_names[:-2]:
    dist_encod = OrdinalEncoder()

    if column in ['Fecha entrega del Informe', 'Tipo de vía','Latitud (Decimal)', 'Longitud (Decimal)', 'Número de frentes', 'Edad', 'Elevador','Área Terreno','Área Construcción', 'Valor comercial (USD)', 'ranva']:
        continue

    enc = dist_encod.fit_transform(ordinal_data[[column,'ranva']].to_numpy(dtype = str))
    ordinal_data[column] = enc.T[0]

ordinal_data = ordinal_data.applymap(lambda x : float(x.replace(',','')) if isinstance(x,str) else x)
ordinal_data = ordinal_data.applymap(lambda x : np.cbrt(x))


In [ ]:
SIMPLE_COLUMNS = [
    'Fecha',
    'TipoDeVia',
    'Piso',
    'Departamento',
    'Provincia',
    'Distrito',
    'Estacionamiento',
    'Depositos',
    'Lat',
    'Lon',
    'Categoria',
    'Posicion',
    'Frentes',
    'Edad',
    'Elevador',
    'Conservacion',
    'Metodo',
    'AreaTerreno',
    'AreaConstruccion',
    'Valorcomercial',
    'ranv'
]

In [ ]:
ordinal_data.columns = SIMPLE_COLUMNS